In [155]:
#import libraries
import pandas as pd

In [156]:
#import datasets
cyberark = pd.read_excel(r'C:\Users\Jarrod\Documents\Data Science\Specialistinterne Workshop\SIEM Task\wbc_siem_cyberark_dataset.xlsx')
unix = pd.read_excel(r'C:\Users\Jarrod\Documents\Data Science\Specialistinterne Workshop\SIEM Task\wbc_siem_unix_auditd_dataset.xlsx')
windows = pd.read_excel(r'C:\Users\Jarrod\Documents\Data Science\Specialistinterne Workshop\SIEM Task\wbc_siem_windows_dataset.xlsx')

In [157]:
#Clean up all date fields
cyberark['Event Time'] = cyberark['Event Time'].str.replace('AEST', '')
cyberark['deviceCustomDate1'] = cyberark['deviceCustomDate1'].str.replace('AEST', '')
cyberark['deviceCustomDate2'] = cyberark['deviceCustomDate2'].str.replace('AEST', '')

unix['Event Time'] = unix['Event Time'].str.replace('AEST', '')
windows['Event Time'] = windows['Event Time'].str.replace('AEST', '')

#Convert all date fields to datetime
cyberark['Event Time'] = pd.to_datetime(cyberark['Event Time'], format='%Y-%m-%d %H:%M:%S')
cyberark['deviceCustomDate1'] = pd.to_datetime(cyberark['deviceCustomDate1'], format='%Y-%m-%d %H:%M:%S')
cyberark['deviceCustomDate2'] = pd.to_datetime(cyberark['deviceCustomDate2'], format='%Y-%m-%d %H:%M:%S')

unix['Event Time'] = pd.to_datetime(unix['Event Time'], format='%Y-%m-%d %H:%M:%S')
windows['Event Time'] = pd.to_datetime(windows['Event Time'], format='%Y-%m-%d %H:%M:%S')

In [158]:
#merge desired columns into one dataframe
merged = pd.concat([unix[['Event Time', 'destinationUserName', 'fDeviceVendor']],\
                    windows[['Event Time', 'destinationUserName', 'fDeviceVendor']]],\
                   axis=0, sort=False)

In [159]:
#Extract only the confirmed approved access from Cyberark dataset
confirmed = cyberark.loc[cyberark['Name'] == 'Retrieve password'].reset_index(drop=True)

In [160]:
#Iterate through and find all authorised access of servers to build a dataframe
approved = pd.DataFrame(columns = [['Event Time', 'destinationUserName', 'fDeviceVendor', 'Name', 'deviceCustomDate1', 'deviceCustomDate2', 'destinationUser_cyberark']])
i = 0
while i < len(confirmed):
    try:
        for index, row in merged.iterrows():
            if (row['Event Time'] > confirmed['deviceCustomDate1'][i]) & (row['Event Time'] <= confirmed['deviceCustomDate2'][i]) & (row['destinationUserName'] == confirmed['destinationUserName'][i]):
                d = list(row[['Event Time', 'destinationUserName', 'fDeviceVendor']])
                k = list(confirmed.loc[i, ['Name', 'deviceCustomDate1', 'deviceCustomDate2', 'destinationUserName']])
                z = d + k
                #Change list items from timestamp to string and add to dataframe
                z[0], z[4], z[5] = str(z[0]), str(z[4]), str(z[5])
                print(z)
                i += 1
                approved.loc[len(approved)] = z
    except Exception as e:
        print(e)
        #break

['2020-09-22 22:00:00', 'dvader', 'Unix', 'Retrieve password', '2020-09-22 21:40:18', '2020-09-22 23:40:18', 'dvader']
['2020-09-21 20:37:22', 'puser04', 'Unix', 'Retrieve password', '2020-09-21 10:00:00', '2020-09-22 00:00:00', 'puser04']
['2020-09-21 20:40:23', 'puser05', 'Unix', 'Retrieve password', '2020-09-21 18:00:00', '2020-09-21 22:00:00', 'puser05']
['2020-09-23 21:00:50', 'dvader', 'Unix', 'Retrieve password', '2020-09-23 20:38:40', '2020-09-23 22:00:00', 'dvader']
['2020-09-22 20:40:20', 'puser05', 'Unix', 'Retrieve password', '2020-09-22 20:00:00', '2020-09-22 22:00:00', 'puser05']
['2020-09-22 21:37:28', 'puser04', 'Unix', 'Retrieve password', '2020-09-22 18:00:00', '2020-09-22 23:00:00', 'puser04']
['2020-09-24 21:29:50', 'dvader', 'Unix', 'Retrieve password', '2020-09-24 20:38:40', '2020-09-24 23:00:00', 'dvader']
['2020-09-23 20:40:23', 'puser05', 'Unix', 'Retrieve password', '2020-09-23 18:00:00', '2020-09-23 23:00:00', 'puser05']
['2020-09-25 21:00:00', 'dvader', 'Uni

In [161]:
#Iterate through and find all unauthorised access of servers
not_approved = pd.DataFrame(columns = [['Event Time', 'destinationUserName', 'fDeviceVendor', 'Name', 'deviceCustomDate1', 'deviceCustomDate2', 'destinationUser_cyberark']])
i = 0
while i < len(confirmed):
    try:
        for index, row in merged.iterrows():
            if not (row['Event Time'] > confirmed['deviceCustomDate1'][i]) & (row['Event Time'] <= confirmed['deviceCustomDate2'][i]) & (row['destinationUserName'] == confirmed['destinationUserName'][i]):
                d = list(row[['Event Time', 'destinationUserName', 'fDeviceVendor']])
                k = list(confirmed.loc[i, ['Name', 'deviceCustomDate1', 'deviceCustomDate2', 'destinationUserName']])
                z = d + k
                #Change list items from timestamp to string and add to dataframe
                #(had trouble adding them as they came out as timestamps)
                z[0], z[4], z[5] = str(z[0]), str(z[4]), str(z[5])
                print(z)
                i += 1
                not_approved.loc[len(not_approved)] = z
    except Exception as e:
        print(e)
        #break

['2020-09-23 21:00:50', 'dvader', 'Unix', 'Retrieve password', '2020-09-22 21:40:18', '2020-09-22 23:40:18', 'dvader']
['2020-09-24 21:29:50', 'dvader', 'Unix', 'Retrieve password', '2020-09-21 10:00:00', '2020-09-22 00:00:00', 'puser04']
['2020-09-25 21:00:00', 'dvader', 'Unix', 'Retrieve password', '2020-09-21 18:00:00', '2020-09-21 22:00:00', 'puser05']
['2020-09-26 23:00:50', 'dvader', 'Unix', 'Retrieve password', '2020-09-23 20:38:40', '2020-09-23 22:00:00', 'dvader']
['2020-09-30 23:00:50', 'dvader', 'Unix', 'Retrieve password', '2020-09-22 20:00:00', '2020-09-22 22:00:00', 'puser05']
['2020-09-21 20:37:22', 'puser04', 'Unix', 'Retrieve password', '2020-09-22 18:00:00', '2020-09-22 23:00:00', 'puser04']
['2020-09-22 21:37:28', 'puser04', 'Unix', 'Retrieve password', '2020-09-24 20:38:40', '2020-09-24 23:00:00', 'dvader']
['2020-09-23 20:37:22', 'puser04', 'Unix', 'Retrieve password', '2020-09-23 18:00:00', '2020-09-23 23:00:00', 'puser05']
['2020-09-24 21:37:20', 'puser04', 'Unix

In [162]:
#print dataframes to csv files 
approved.to_csv('approved_server_use.csv')
not_approved.to_csv('not_approved_server_use.csv')